In [ ]:
import msmrd2
from msmrd2.integrators import langevin
from msmrd2.potentials import WCA, gaussians3D
import msmrd2.tools.particleTools as particleTools
import msmrd2.tools.analysis as analysisTools

import matplotlib.pyplot as plt
import numpy as np
import h5py
import random
import sys
import os

In [ ]:
## Units

# ### Boltzman constant
# - $k_B = 1.38064852 \times 10^{-23} \frac{m^2}{s^2} \frac{kg}{K} \left(= \frac{nm^2}{ns^2}\frac{kg}{K}\right)$.
#
# ### Basic units
# - Length (l): nanometers (nm)
# - Energy ($\epsilon$) : $k_B T = g/mol \frac{nm^2}{ns^2}$
# - Mass (m): gram/mol (g/mol)
#
# ### Derived units
# - Time: $l\sqrt{m/\epsilon} = ns$
# - Temperature: $\epsilon/k_B =$ Kelvin ($K$)
# - Force: $\epsilon/l = kg \frac{nm}{ns^2}$
#
# ### Reduced quantities (dimensionless)
# - Reduced pair potential: $U^* = U/\epsilon$
# - Reduced force: $F^* = F l /\epsilon$
# - Reduced distance: $r^* = r/l$
# - Reduced density: $\rho^*=\rho l^3$
# - Reduced Temperature: $T^* = k_B T/\epsilon$
# - Reduced Pressure: $P^* = Pl^3/\epsilon$
# - Reduced friction: $\sigma^2/time$

In [ ]:
# Main parameters
numBathParticles = 100 
numparticles = 1 + numBathParticles #Added distinguished particle (index 0)
D = 3.0E-2 #3.0E-2 #3.0E-2 #1.0E-3 #(nm^2/ns) Note 1.0E-3 nm^2/ns = 1 micrometer^2/s #0.1
particlemass = 18.0 # (g/mol) approximately mass of water
distinguishedParticleMass = 3 * particlemass # (kg)
particleDiameter = 0.3 # (nm)
separationDistance = 2 * particleDiameter # minimum separation distance for initial condition
# For computations, we assume KbT=1, thus the force F must be: F=KbT f, where f is the force computed
# from the potential. This means the plotted potential is on reduced units (not the distances though);
KbT = 0.0000

# Main parameters for integrator
dt = 0.005 #0.005 #0.001 #0.0005 # (ns)
seed = -1 # Seed = -1 used random device as seed
bodytype = 'point'

# Define simulation boundaries (choose either spherical or box)
boxsize = 8 #(nm)
boundaryType = 'periodic'

# Parameters for WCA potential (rm=2^(1/6)sigma)
epsilon = 1.0
rm = particleDiameter # (diameter in nm)
sigma = rm * 2**(-1/6)

# Parameters for external potential (will only acts on distinguished particles (type 1))
minimas = np.array([[-1.5,0,0], [1.5,0,0]])
sigma = 1
standardDevs = np.array([[sigma, sigma, sigma], [sigma, sigma, sigma]])
scalefactor = 75

# Simulation parameters
tfinal = 10 #50.0 #50.0
timesteps = int(tfinal/dt) 
bufferSize = 100 * 1024
stride = 5 
outTxt = False
outH5 = True
outChunked = True
outputEnergyTempTrajectory = True
trajtype = "positionVelocityDistinguished" # Samples position and velocity of distinguished particle (type 1) + raux variables
distinguishedTypes = [1] # Only saves positions of particle with these types
equilibrationSteps = 0 #2000

In [ ]:
# Parent directory location
parentDirectory = "./testsData/"

# Create folder for data
try:
    os.mkdir(parentDirectory)
except OSError as error:
    proceed = True

basefilename = os.path.join(parentDirectory, "simulationDistinguished")

In [ ]:
# Run one simulation and saves data on H5 file

# Define particle list
partlist = particleTools.randomLangevinParticleList(numparticles, boxsize, separationDistance, D,
                                                    particlemass, seed, distinguishedParticleOrigin=True)
# Set distinguished particle (default type is zero)
partlist[0].setType(1)
partlist[0].setMass(distinguishedParticleMass)

# Define boundary
boxBoundary = msmrd2.box(boxsize, boxsize, boxsize, boundaryType)

# Define pair potential
potentialWCA = WCA(epsilon, sigma)
potentialWCA.setForceCapValue(100)

# Define external potential
#externalPotential = gaussians3D(minimas, standardDevs, distinguishedTypes, scalefactor)

# Integrator definition
seed = int(-1) 
integrator = langevin(dt, seed, bodytype)
integrator.setBoundary(boxBoundary)
integrator.setPairPotential(potentialWCA)
#integrator.setExternalPotential(externalPotential)
integrator.setKbT(KbT)

# Creates simulation
sim = msmrd2.simulation(integrator)

# Runs simulation
sim.setEquilibrationSteps(equilibrationSteps)
sim.setOutputEnergyTemperature(outputEnergyTempTrajectory) 
sim.setDistinguishedTypes(distinguishedTypes)

sim.run(partlist, timesteps, stride, bufferSize, basefilename, outTxt, outH5, outChunked, trajtype)
print("Simulation done")

In [ ]:
#potentialWCA.setForceCapValue(0.1)


In [ ]:
potentialWCA.getPotentialCapValue()

In [ ]:
#potentialWCA = WCA(epsilon, sigma)
rr = 2**(1/6)
drr = rr/2000
part1 = msmrd2.particle(0,0,0,0, [0,0,0], [1,0,0,0])
part2 = msmrd2.particle(0,0,0,0, [0.9063881040098023,0,0], [1,0,0,0])
np.linalg.norm(potentialWCA.forceTorque(part1,part2)[0])

In [ ]:
potentialWCA.evaluate(part1,part2)

In [ ]:
# Read H5 file for the energy and temperature
f = h5py.File(basefilename + '_energytemp.h5', 'r')

# List all groups (use the firs key)
print("Keys: %s" % list(f.keys()))
a_group_key = list(f.keys())[0]

# Get the data (different possibilities)
dataET = f[a_group_key]

In [ ]:
# Plot energy and temperature
time = dataET[:,0]
time = time - time[0]
kineticEnergy = dataET[:,1]
potentialEnergy = dataET[:,2]
totalEnergy = dataET[:,3]
temperature = dataET[:,4]
plt.plot(time, totalEnergy)
plt.plot(time, kineticEnergy)
plt.xlabel('time(ns)')
plt.ylabel('energy')

In [ ]:
plt.plot(time, potentialEnergy)

In [ ]:
plt.plot(time, temperature)
plt.xlabel('time(ns)')
plt.ylabel('temperature')

In [ ]:
# Read H5 file for distinguished particle position
f2 = h5py.File(basefilename + '.h5', 'r')

# List all groups (use the firs key)
print("Keys: %s" % list(f2.keys()))
a_group_key2 = list(f2.keys())[0]

# Get the data (different possibilities)
data = f2[a_group_key2]

In [ ]:
# Plot distinguished particle position
positionx = data[:,1]
positiony = data[:,2]
positionz = data[:,3]
plt.plot(positionx, positionz)

In [ ]:
# NEED TO CHECK UNITS OF KINETIC ENERGY. AT THE MOMENT KINETIC ENERGY DOMINATES COMPLETELY THE SUMS OF THE ENERGIES
